## 0. Simulate some data and fit emulator

In [1]:
import torch

from autoemulate.experimental.simulations.projectile import ProjectileMultioutput
from autoemulate.experimental.emulators.gaussian_process.exact import (
    GaussianProcessExact,
)

In [2]:
sim = ProjectileMultioutput()
x = sim.sample_inputs(10)
y = sim.forward_batch(x)

Running simulations: 100%|██████████| 10/10 [00:00<00:00, 843.25it/s]

Successfully completed 10/10 simulations (100.0%)


In [3]:
# check range of training outputs
torch.min(y, dim=0).values, torch.max(y, dim=0).values

(tensor([1.1038, 1.4069], dtype=torch.float64),
 tensor([17915.9616,   305.6047], dtype=torch.float64))

In [4]:
gp = GaussianProcessExact(x, y)
gp.fit(x, y)

## 1. Simple HMC example.

In [5]:
from autoemulate.experimental.calibration.hmc import HMCCalibrator

Lets start with an observation inside the training range, we should be able to recover the input parameters.

In [6]:
# given the ranges, we can actually just them to the same thing
observations = {name: 100 for name in sim.output_names}

In [7]:
hmc = HMCCalibrator(gp, sim.parameters_range, observations, 1.0)

In [8]:
mcmc = hmc.run_mcmc(warmup_steps=10, num_samples=100)

Sample: 100%|██████████| 110/110 [00:00, 256.07it/s, step size=1.30e-02, acc. prob=0.022]


The returned Pyro MCMC object has methods for accessing the generated samples (`mcmc.get_samples()`) or, as shown below, to get just their summary statistics.

In [9]:
mcmc.summary()


                mean       std    median      5.0%     95.0%     n_eff     r_hat
         c     -3.18      0.06     -3.14     -3.25     -3.11      2.72      2.46
        v0    702.19      0.15    702.10    702.09    702.33      5.19      1.37

Number of divergences: 100


## 2. Combining this with sensitivity analysis and history matching.